# Metrics Quickstart

Compute Tier I technical indicators using the in-repo metrics package.

## Load sample data

In [1]:
from pathlib import Path
import sys

try:
    ROOT = Path(__file__).resolve().parents[1]
except NameError:
    ROOT = Path.cwd()
    if not (ROOT / "metrics").exists() and (ROOT.parent / "metrics").exists():
        ROOT = ROOT.parent

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

In [3]:
import pandas as pd
from metrics import compute_metrics, list_metrics

df = pd.read_csv("../tests/data/btc_ohlcv_sample.csv", parse_dates=["timestamp"])
df.head()

,timestamp,open,high,low,close,volume
0,2024-01-01 00:00:00+00:00,39983.420100,40048.156625,39952.686712,40029.802849,931.391588
1,2024-01-01 01:00:00+00:00,40018.590881,40079.312074,39964.193828,40021.500810,314.381558
2,2024-01-01 02:00:00+00:00,40036.454716,40118.072890,40018.978492,40060.383011,1042.941011
3,2024-01-01 03:00:00+00:00,40072.608844,40203.389345,39996.473999,40151.902750,438.454504
4,2024-01-01 04:00:00+00:00,40151.483131,40207.209249,NaN,40137.800195,662.776549


## Available metrics

In [4]:
list_metrics()

['ADX',
 'ATR',
 'BollingerBands',
 'EMA',
 'MACD',
 'OBV',
 'ROC',
 'RSI',
 'SMA',
 'VWAP',
 'WMA']

## Compute indicators (wide format)

In [5]:
signals = compute_metrics(
    df,
    features=["RSI", "MACD", "BollingerBands", "ATR", "VWAP", "ADX"],
    params={"RSI": {"period": 14}, "BollingerBands": {"period": 20, "mult": 2.0}},
    output="wide"
)
signals.filter(regex="RSI|MACD|BBANDS|ATR|VWAP|ADX").head()

,RSI_14,MACD_12_26_9,MACD_12_26_9_signal,MACD_12_26_9_hist,BBANDS_20_2.0_basis,BBANDS_20_2.0_upper,BBANDS_20_2.0_lower,BBANDS_20_2.0_bandwidth,BBANDS_20_2.0_pctB,ATR_14,VWAP,ADX_14_plus_di,ADX_14_minus_di,ADX_14
0,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.0,0.0,NaN,40010.215396,NaN,NaN,NaN
1,NaN,-0.662271,-0.132454,-0.529817,NaN,NaN,NaN,0.0,0.0,NaN,40013.105787,NaN,NaN,NaN
2,NaN,1.928117,0.279660,1.648457,NaN,NaN,NaN,0.0,0.0,NaN,40037.123165,NaN,NaN,NaN
3,NaN,11.236371,2.471002,8.765368,NaN,NaN,NaN,0.0,0.0,NaN,40050.006242,NaN,NaN,NaN
4,NaN,17.276122,5.432026,11.844096,NaN,NaN,NaN,0.0,0.0,NaN,0.000000,NaN,NaN,NaN


## Tidy output for LLM ingestion

In [9]:
tidy = compute_metrics(df, features=["MACD", "BollingerBands"], output="long")
tidy.tail(20)

,timestamp,feature,key,value
2380,2024-01-13 09:00:00+00:00,BollingerBands,upper,39857.372383
2381,2024-01-13 09:00:00+00:00,MACD,hist,6.746227
2382,2024-01-13 09:00:00+00:00,MACD,signal,-16.787964
2383,2024-01-13 09:00:00+00:00,MACD,value,-10.041738
2384,2024-01-13 10:00:00+00:00,BollingerBands,bandwidth,0.004943
2385,2024-01-13 10:00:00+00:00,BollingerBands,basis,39770.180764
2386,2024-01-13 10:00:00+00:00,BollingerBands,lower,39671.882389
2387,2024-01-13 10:00:00+00:00,BollingerBands,pctB,0.904062
2388,2024-01-13 10:00:00+00:00,BollingerBands,upper,39868.479139
2389,2024-01-13 10:00:00+00:00,MACD,hist,11.347067


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   timestamp  300 non-null    datetime64[ns, UTC]
 1   open       300 non-null    float64            
 2   high       297 non-null    float64            
 3   low        297 non-null    float64            
 4   close      300 non-null    float64            
 5   volume     300 non-null    float64            
dtypes: datetime64[ns, UTC](1), float64(5)
memory usage: 14.2 KB
